In [ ]:
#solar data -> might be used later on
#data_solar = pd.read_table('data/stundenwerte_ST_05100_row/produkt_st_stunde_19580101_20220531_05100.txt', sep=';')
#data_solar = data_solar.drop('eor', axis=1)
#data_solar.MESS_DATUM = pd.to_datetime(data_solar.MESS_DATUM.astype(str), format=('%Y%m%d%H:%M'))
#data_solar[data_solar == -999] = np.nan
#data_solar = data_solar[data_solar.MESS_DATUM >= pd.to_datetime('2019-01-01')]
#data_solar.reset_index(drop = True, inplace=True)

In [ ]:
#not needed yet
#px.line(data_solar, x='MESS_DATUM', y='FG_LBERG')

In [ ]:
def energy_needed(df): #returns dataframe with needed energy for preheating and dataframe with periods
    df_periods = pd.DataFrame(columns=['period', 'en_needed_water', 'en_needed_elec', 'en_needed_elec_preheat', 'stor_vol_needed'])
    for p in pd.Series(df.period_max.unique()).dropna():
        period = df[df.period_max == p].reset_index(drop=True)
        eta_p = eta_freezing_adj(zu = period.temperature, max_ab=max(period.vorlauf_temp))
        eta_p_freezing_adjusted = eta_heating(zu = period.temperature, max_ab=max(period.vorlauf_temp))
        eta_p_freezing_adjusted[(period.freezing_zone == True)] = eta_p[(period.freezing_zone == True)] / 1.2

        integ_water = integral(period.index, period.energy_needed_water)
        integ_elec = integral(period.index, period.energy_needed_elec)

        integ_water_profit = integral(period[eta_p[0] > period.eta_hp_heating].index, period[eta_p[0] > period.eta_hp_heating].energy_needed_water)
        integ_elec_profit = integral(period[eta_p[0] > period.eta_hp_heating].index, period[eta_p[0] > period.eta_hp_heating].energy_needed_elec)

        vol_needed = get_needed_storage_vol(max(period.vorlauf_temp) - ruecklauf, integ_water / 1000)
        vol_needed_profit = max(0, get_needed_storage_vol(max(period.vorlauf_temp) - ruecklauf, integ_water_profit / 1000))

        df.loc[df.period_max == p, 'energy_needed_water_total_period'] = integ_water
        df.loc[df.period_max == p, 'energy_needed_elec_total_period'] = integ_elec
        df.loc[df.period_max == p, 'storage_vol_needed'] = vol_needed
        df.loc[df.period_max == p, 'eta_hp_heating_nextmin'] = list(eta_p)
        df.loc[df.period_max == p, 'eta_hp_heating_nextmin_freezing_adjusted'] = list(eta_p_freezing_adjusted)
        df.loc[df.period_max == p, 'preheat_profitable'] = list(eta_p[0] > period.eta_hp_heating)
        df.loc[df.period_max == p, 'preheat_profitable_freezing_adjusted'] = list(eta_p_freezing_adjusted[0] > period.eta_freezing_adjusted)
        df.loc[(df.period_max == p) & (df.preheat_profitable == True), 'energy_needed_water_profitable_period'] = integ_water_profit
        df.loc[(df.period_max == p) & (df.preheat_profitable == True), 'energy_needed_elec_profitable_period'] = integ_elec_profit

        df_periods = df_periods.append({'period': p,
                                        'en_needed_water': integ_water,
                                        'en_needed_elec': integ_elec,
                                        'en_needed_elec_profit': integ_elec_profit,
                                        'en_needed_water_profit': integ_water_profit,
                                        'en_needed_elec_preheat': integ_water / eta_p[0],
                                        'en_needed_elec_preheat_profit': integ_water_profit / eta_p[0],
                                        'stor_vol_needed': vol_needed,
                                        'stor_vol_needed_profit': vol_needed_profit}, ignore_index=True)

    df.loc[df.preheat_profitable == False, 'en_produced_hourly'] = df.loc[df.preheat_profitable == False, 'eta_hp_heating_nextmin'] * hp_power_el_opt
    df.loc[df.preheat_profitable == True, 'en_produced_hourly'] = 0

    df['en_produced_total'] = df.en_produced_hourly.cumsum()

    return df, df_periods
data_temp, data_temp_periods = energy_needed(data_temp)

In [ ]:
for p in pd.Series(data_temp.period_min.unique()).dropna():

    data_temp.loc[data_temp.period_min == p, 'energy_overheat'] = energy_total_overheat(groundwater, data_temp.loc[data_temp.period_min == p, 'vorlauf_temp'].values[-1], data_temp.loc[data_temp.period_min == p, 'eta_hp_heating'].values, np.flip(data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index))
    data_temp.loc[data_temp.period_min == p, 'energy_keepheat'] = energy_total_keepheat(groundwater, data_temp.loc[data_temp.period_min == p, 'vorlauf_temp'].values[-1], data_temp.loc[data_temp.period_min == p, 'eta_hp_heating'].values, np.flip(np.cumsum(np.flip(data_temp.loc[data_temp.period_min == p, 'eta_hp_heating'].values)) / (data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index+1)),  np.flip(data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index))
    temp_opt, en_opt, power_opt = over_keep_mix_vectorized(time = np.flip(data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index), vorlauftemp = data_temp.loc[data_temp.period_min == p, 'vorlauf_temp'].values[-1], eta_t_0 = data_temp.loc[data_temp.period_min == p, 'eta_hp_heating'].values, eta_avg=np.flip(np.cumsum(np.flip(data_temp.loc[data_temp.period_min == p, 'eta_hp_heating'].values)) / (data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index+1)))
    #print(temp_opt.shape, en_opt.shape, power_opt.shape)
    data_temp.loc[data_temp.period_min == p, ['temp_0_optimized', 'energy_optimized', 'storage_power_optimized']] = np.array([temp_opt, en_opt, power_opt]).reshape(3, temp_opt.shape[0]).T
    data_temp.loc[data_temp.period_min == p, 'temp_0'] = celsius(newton_cooling_t_0(kelvin(data_temp.loc[data_temp.period_min == p, 'vorlauf_temp'].values[-1]), np.flip(data_temp.loc[data_temp.period_min == p, :].reset_index(drop=True).index.values) * 3600))


In [ ]:
#def optimal_heating(df):
#    per_min = df[df.period_min == 1] #period from miminum to minimum
#    per_max = df[df.period_max == 1] #period from max to max
#
#    max_vorlauf = per_max.local_max_vorlauf.dropna().values[0]
#    max_vorlauf_index = per_max.local_max_vorlauf.dropna().index.values
#
#    per_min['timedelta_min'] = max_vorlauf_index - per_min.index.values
#    vorlauf = newton_cooling_t_0(max_vorlauf, per_min.timedelta_min)
#    #per_min['eta_hp_heating_nextmin'] = eta_heating(zu = per_min.temperature, max_ab = vorlauf)
#
#    energy_needed_water = per_max[per_max.preheat_profitable == True].energy_needed_water.sum()
#    print('Energy needed for period: ', energy_needed_water)
#    per_min_sorted = per_min.sort_values(by='eta_hp_heating_nextmin_freezing_adjusted', ascending=False)
#    eta_cumsum = per_min_sorted.eta_hp_heating_nextmin_freezing_adjusted.cumsum()
#    eta_cumsum_shift = eta_cumsum - per_min_sorted.eta_hp_heating_nextmin_freezing_adjusted
#    #print(eta_cumsum_shift)
#    per_min_sorted['heating'] = eta_cumsum_shift < energy_needed_water / hp_power_el_opt
#    per_min = per_min_sorted.sort_values(by = 'date')
#
#    #per_min.loc[((per_min.preheat_profitable == False) & (per_min.heating == True)), 'en_produced_hourly'] = per_min.loc[((per_min.preheat_profitable == False) & (per_min.heating == True)), 'eta_hp_heating_nextmin'] * hp_power_el_opt
#    per_min.loc[(per_min.heating == True), 'en_produced_hourly'] = per_min.loc[(per_min.heating == True), 'eta_hp_heating_nextmin'] * hp_power_el_opt
#    #per_min.loc[((per_min.preheat_profitable == True) | (per_min.heating == False)), 'en_produced_hourly'] = 0
#    per_min.loc[(per_min.heating != True), 'en_produced_hourly'] = 0
#
#    per_min['en_produced_total'] = per_min.en_produced_hourly.cumsum()
#    per_min = pd.concat((per_min, per_max[per_max.period_min != 1]))
#    return per_min
#per1 = optimal_heating(forecast)

In [ ]:
#def optimal_heating2(df):
#    p = 1
#    df_per = df[(df.period_max == p) | (df.period_min == p)]
#    per_min = df[df.period_min == p] #period from miminum to minimum
#    per_max = df[df.period_max == p] #period from max to max
#
#    max_vorlauf = per_max.local_max_vorlauf.dropna().values[0]
#    max_vorlauf_index = per_max.local_max_vorlauf.dropna().index.values
#
#    df_per.loc[(df_per.period_min == p), 'timedelta_min'] = max_vorlauf_index - per_min.index.values
#    vorlauf = celsius(newton_cooling_t_0(kelvin(max_vorlauf), df_per.timedelta_min.values * 3600))
#
#    # efficiency based on temperature and needed preheating temperature, s.t. temp is at max_vorlauf
#    df_per.loc[(df_per.period_min == p), 'vorlauf_temp_nextmin'] = pd.Series(vorlauf).dropna().values
#    df_per.loc[(df_per.period_min == p), 'eta_hp_heating_nextmin'] = eta_heating(zu = per_min.temperature.values, max_ab = pd.Series(vorlauf).dropna().values)
#    #df_per.loc[(df_per.period_min == p), 'preheat_profitable'] =
#    return df_per
#
#per2 = optimal_heating2(forecast)

In [ ]:
periods = 48
def heating_opt3(df, p = 2):
    #df_per = df[(df.period_max == p) | (df.period_min == p)]
    df_per = df#[(df.period_max == p) | (df.period_min == p)]
    per_min = df[df.period_min == p] #period from miminum to minimum
    per_max = df[df.period_max == p] #period from max to max

    per = df_per.shape[0] #number of hours in period
    M_timedelta = np.empty((per, per))

    for i in range(per): # build timedelta matrix
        M_timedelta[i, range(0, i+1)] = np.flip(np.linspace(0, i, i+1))
    M_timedelta = M_timedelta.T
    vorlauf = df_per.vorlauf_temp.values

    M_vorlauf = celsius(newton_cooling_t_0(kelvin(vorlauf), M_timedelta * 3600))
    M_vorlauf = np.triu(M_vorlauf) # needed heating temperature when heating in period i and using the heat in period j (i X j - Matrix)
    M_temperature = np.tile(df_per.temperature.values, (len(df_per.temperature.values),1)).T

    M_eta = eta_freezing_adj(zu=M_temperature, max_ab=M_vorlauf)#, min_ab=M_vorlauf)

    M_vorlauf_max = celsius(newton_cooling_t_0(kelvin(vorlauf_max), t=M_timedelta * 3600))
    M_eta_heating = eta_freezing_adj(zu = M_temperature, max_ab=M_vorlauf_max, heating=True)
    print(M_vorlauf_max)
    M_eta_heating = np.triu(M_eta_heating, k=1)
    M_eta = np.tril(M_eta)

    M_eta = M_eta_heating + M_eta
    M_eta = np.triu(M_eta) # upper diagonal
    M_eta = np.tril(M_eta, k = periods)

    P_w_sum = df_per.energy_needed_water.values # needed energy in water for each hour
    P_e_max = np.full((per, ), hp_power_el_opt) # maximal available energy (electricity) per hour -> e.g. 2500W
    P_e_available = P_e_max.copy() # at each period still available 'capacity' in electricity for future hours

    P_e = np.zeros((per, per)) # Matrix with used energy (e) from each period for each period that is optimized
    P_e_a = np.empty((per, per)) # Matrix to later calculate P_e

    for j in range(per):
        P_w_j = P_w_sum[j] # energy needed (w) in hour j

        M_eta_j = M_eta[: , j] # efficiencies in each hour
        eta_sorted_j_index = np.flip(np.argsort(M_eta_j)) # get indexes of highest values in M_eta_j
        eta_sorted_j = M_eta_j[eta_sorted_j_index] # get highest values of M_eta_j (descending order)

        P_w_available_j = P_e_available[eta_sorted_j_index] * eta_sorted_j # available energy (w) for hour j in descending cost order

        P_w_available_j_cumsum = P_w_available_j.cumsum() # get cumsum for available P_w (descending cost order)
        P_w_available_j_cumsum_shift = P_w_available_j_cumsum - P_w_available_j # shifted cumsum

        # calculate which P_w_available_j values are used fully and which only partly (only one will be used partly)
        cumsum_bin = P_w_available_j_cumsum < P_w_j # binary values if cumsum is smaller than needed energy
        cumsum_shift_bin = P_w_available_j_cumsum_shift < P_w_j # binary values if cumsum_shift is smaller than needed energy

        idx = (cumsum_bin == False) & (cumsum_shift_bin == True) # only True for partly used P_w_j
        percent = (P_w_available_j_cumsum[idx] -P_w_j)  / P_w_available_j[idx] # get remaining share of partly used P_w_j
        P_e_percent_remaining = np.ones((per, )) # first, all remaining values are set to 1 (=100%)
        P_e_percent_remaining[cumsum_bin] = 0 # set all fully used P_w_j to 0
        P_e_percent_remaining[idx] = percent # set partly used to calculated percentage

        P_e_percent_remaining_sorted_orig = P_e_percent_remaining[np.argsort(eta_sorted_j_index)] # set remaining shares to original order
        P_e_available = P_e_available * P_e_percent_remaining_sorted_orig # get new P_e_available

        P_e_a[:,j] = P_e_available

    P_e_a = np.triu(P_e_a) # take only upper diagonal
    P_e[:,0] = P_e_a[:,0]
    for j in range(1, per):
        P_e[:,j] = np.abs(P_e_a[:,j] - P_e_a[:,j-1])
    diagonal = np.zeros((per, per))
    np.fill_diagonal(diagonal, hp_power_el_opt)

    P_e = np.abs(diagonal - P_e)
    P_w = (P_e * M_eta)

    df_per['P_e_opt'] = P_e.sum(1)

    return   M_eta, P_e, P_w, df_per

In [ ]:

#fig = make_subplots(rows=78, cols=3)
#
#stor_vol = []
#for period in range(78):
#    print('Period: ', period)
#    M_eta1, P_e1,  P_w1, df_per1 = heating_opt(data_temp, p= period)
#    print(P_e1.sum() / df_per1.energy_needed_elec.sum())
#
#    df_per1['P_e_opt'] = P_e1.sum(1)
#    P_w1 = (P_e1 * M_eta1)
#
#    stored_water_energy1 = np.zeros((P_w1.shape[0], ))
#    for i in range(P_w1.shape[0]-2):
#
#        stored_water_energy1[i] = P_w1[:(i+1), (i+1):].sum()
#    stor_vol.append(max(get_needed_storage_vol(10, stored_water_energy1 / 1000)))
#    print('storage_volume: ', max(get_needed_storage_vol(10, stored_water_energy1 / 1000)))
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.temperature, name='temperature', mode='lines', line=dict(color='blue')), row=period +1, col=1)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.local_min, mode='markers', marker=dict(color='orange')),row = period + 1, col = 1)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.local_max, mode='markers', marker=dict(color='lightgreen')),row = period + 1, col = 1)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.P_e_opt, name='Power (e) optimized', mode='markers', line=dict(color='red')), row=period +1, col=2)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.energy_needed_elec, name='Power (e) not optimized', mode='markers'),  row=period+1, col=2)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = df_per1.energy_needed_water.cumsum(), name='power needed (water)', mode='lines', line=dict(color='red')), row=period+1, col=3)
#    fig.add_trace(go.Scatter(x = df_per1.date, y = P_w1.sum(1).cumsum(), name='optimized power (water)', mode='lines', line=dict(color='green')), row=period+1, col=3)
#
#fig.update_layout(height= 8000, showlegend = False)
##fig.show()

In [ ]:
def constraint_fun(vorl):

        #M_eta = eta_freezing_adj(zu=M_temperature, max_ab=M_vorlauf)#, min_ab=M_vorlauf)
#
        #if type(period_lim) != type(None):
        #    M_vorlauf_max = celsius(newton_cooling_t_0(kelvin(vorl), t=M_timedelta * 3600))
        #    M_eta_heating = eta_freezing_adj(zu = M_temperature, max_ab=M_vorlauf_max, heating=False)
        #    M_eta_heating = np.triu(M_eta_heating, k=1)
        #    M_eta = np.tril(M_eta)
#
        #    M_eta = M_eta_heating + M_eta
        #    M_eta = np.triu(M_eta) # upper diagonal
        #    M_eta = np.tril(M_eta, k = period_lim)
#
        #else:
        #    M_eta = np.triu(M_eta) # upper diagonal
#
        #P_w_sum = df_per.energy_needed_water.values # needed energy in water for each hour
        #P_e_max = np.full((per, ), hp_power_el_opt) # maximal available energy (electricity) per hour -> e.g. 2500W
        #P_e_available = P_e_max.copy() # at each period still available 'capacity' in electricity for future hours
#
        #P_e = np.zeros((per, per)) # Matrix with used energy (e) from each period for each period that is optimized
        #P_e_a = np.empty((per, per)) # Matrix to later calculate P_e
#
        #for j in range(per):
        #    P_w_j = P_w_sum[j] # energy needed (w) in hour j
#
        #    M_eta_j = M_eta[: , j] # efficiencies in each hour
        #    eta_sorted_j_index = np.flip(np.argsort(M_eta_j)) # get indexes of highes values in M_eta_j
        #    eta_sorted_j = M_eta_j[eta_sorted_j_index] # get highest values of M_eta_j (descending order)
#
        #    P_w_available_j = P_e_available[eta_sorted_j_index] * eta_sorted_j # available energy (w) for hour j in descending cost order
#
        #    P_w_available_j_cumsum = P_w_available_j.cumsum() # get cumsum for available P_w (descending cost order)
        #    P_w_available_j_cumsum_shift = P_w_available_j_cumsum - P_w_available_j # shifted cumsum
#
        #    # calculate which P_w_available_j values are used fully and which only partly (only one will be used partly)
        #    cumsum_bin = P_w_available_j_cumsum < P_w_j # binary values if cumsum is smaller than needed energy
        #    cumsum_shift_bin = P_w_available_j_cumsum_shift < P_w_j # binary values if cumsum_shift is smaller than needed energy
#
        #    idx = (cumsum_bin == False) & (cumsum_shift_bin == True) # only True for partly used P_w_j
        #    percent = (P_w_available_j_cumsum[idx] -P_w_j)  / P_w_available_j[idx] # get remaining share of partly used P_w_j
        #    P_e_percent_remaining = np.ones((per, )) # first, all remaining values are set to 1 (=100%)
        #    P_e_percent_remaining[cumsum_bin] = 0 # set all fully used P_w_j to 0
        #    P_e_percent_remaining[idx] = percent # set partly used to calculated percentage
#
        #    P_e_percent_remaining_sorted_orig = P_e_percent_remaining[np.argsort(eta_sorted_j_index)] # set remaining shares to original order
        #    P_e_available = P_e_available * P_e_percent_remaining_sorted_orig # get new P_e_available
        #    P_e_a[:,j] = P_e_available
#
        #P_e_a = np.triu(P_e_a) # take only upper diagonal
        #P_e[:,0] = P_e_a[:,0]
        #for j in range(1, per):
        #    P_e[:,j] = np.abs(P_e_a[:,j] - P_e_a[:,j-1])
        #diagonal = np.zeros((per, per))
        #np.fill_diagonal(diagonal, hp_power_el_opt)
#
        #P_e = np.abs(diagonal - P_e)
        #P_w = (P_e * M_eta)